# Histogram of Ordered Gradients (HOG) Demo

HOG first appeared in a patent application from Robert K. McConnell in 1981.  It was later leveraged by Mitsubishi Electric Research in 1994.  The current resurgence of HOG is primarily focused on a paper by Navneet Dalal & Bill Triggs presented at CVPR in 2005.  You can see the link to this paper below: 

Paper: [Histograms of Oriented Gradients for Human Detection](https://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf)

## Install

We will install the following modules that will be used throughout this notebook.  Execute the following cell to install the needed dependencies.


In [ ]:
!pip install opencv-contrib-python==3.3.0.9 imutils scikit-image

## Imports & Utility Functions

Next, we will need to import libraries and add a utility function that we will use throughout the notebook:

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2 as cv
from skimage.feature import hog
from skimage import data, exposure, io
from imutils.object_detection import non_max_suppression
from enum import Enum

class ImageColor(Enum):
    GRAYSCALE = 0
    DEFAULT = 1
    BGR = 2

# This is a utility function that we will use to display images throughout the notebook
def showImage(cv_image, image_color, title = ''):
    plt.figure(figsize=(20,20))
    if image_color == ImageColor.GRAYSCALE:
        plt.imshow(cv_image, cmap = plt.cm.gray)
    elif image_color == ImageColor.BGR:
        plt.imshow(cv.cvtColor(cv_image, cv.COLOR_BGR2RGB))
    else:
        plt.imshow(cv_image)
    if title:
        plt.title(title)
    plt.show()

## Our Objective

We will be working to accomplish two primary tasks: visualizing the HOG feature vector for a specific image, and utilizing the included person detector in OpenCV to see HOG at work. For the visualization of the HOG feature vector we will be leveraging `scikit-image`. 

Learn More: [scikit-image](https://scikit-image.org/)

![Person Image](person1.jpg)

## Visualizing HOG

In the following example, we will utilize a feature of `scikit-image` to visualize the Histogram of Ordered Gradients feature vector for the entire image.

In [ ]:
def visualize_hog(filename):
    # Read the image into a format scikit-image can work with
    image = io.imread(filename)
    
    # scikit-image HOG parameters
    orientations = 8
    pixels_per_cell = (16,16)
    cells_per_block=(1, 1)
    
    # Get vector and HOG image
    feature_vector, hog_image = hog(image,
                                    orientations=orientations,
                                    pixels_per_cell=pixels_per_cell,
                                    cells_per_block=cells_per_block,
                                    visualize=True,
                                    multichannel=True)
    
    # Rescale intensity based on vector values
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    
    # Show images
    showImage(image, False, 'Original Image')
    showImage(hog_image_rescaled, ImageColor.GRAYSCALE, 'Histogram of Ordered Gradients Image')

We will now utilize this function to visualize the feature vector for this image:

In [ ]:
visualize_hog('person1.jpg')

## Detecting People

Up to this point, we have leveraged a non-learning based approach for the images we are analyzing.  This next objective will bridge the gap between the non-learning and the learning based approach.  OpenCV includes a person detector.  This detector utilizes HOD vectors for analysis, but it includes a pre-trained Linear Support Vector Machine (SVN) model. 

This is an example that illustrates how we can use the outputs of algorithms from a non-learning approach to train a model using a learning based approach.

Given the training data that would be needed to train our own model, we will instead use the included detector from OpenCV.

We will now create a function that will attempt to detect people in the images was pass into it.  We will use this function to analyze this functionality across multiple images.

In [ ]:
def detect_people(filename):
    # Setup HOG and detector
    hog_cv = cv.HOGDescriptor()
    hog_cv.setSVMDetector(cv.HOGDescriptor_getDefaultPeopleDetector())

    # Get image in a format OpenCV can work with
    image = cv.imread(filename)
    
    # HOG Detector Parameters
    stride = (4, 4)
    padding = (8, 8)
    scale = 1.05

    # Get detected weights and rects for matches
    rects, weights = hog_cv.detectMultiScale(image,
                                             winStride=stride,
                                             padding=padding,
                                             scale=scale)

    # Convert xywh to xyxy rects for suppression function
    converted_rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])

    # Suppress all rects that are contained within another rect
    final_matches = non_max_suppression(converted_rects,
                                        probs=None,
                                        overlapThresh=0.65)

    # Draw a Rectangle around the detected people
    rect_color = (0, 255, 0)
    for (xA, yA, xB, yB) in final_matches:
        cv.rectangle(image,
                     (xA, yA),
                     (xB, yB),
                     rect_color,
                     2)
    
    # Show the image with drawn rects
    num_matches = len(final_matches)
    showImage(image, ImageColor.BGR, f"{num_matches} Detected Person{'s' if num_matches > 1 else ''}")

In [ ]:
detect_people('person1.jpg')

We will now define a function that allows us to wrap the HOG visualization and the person detection into a single call:

In [ ]:
def visualize_detect_people(filename):
    visualize_hog(filename)
    detect_people(filename)

In [ ]:
visualize_detect_people('people1.jpg')

In [ ]:
visualize_detect_people('people2.jpg')